# Control Systems 1, NB07: Time-Domain specifications and PID

© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook is designed to introduce fundamental concepts in control systems engineering. It covers the steady-state error, different Time-Domain Specifications for first and second order systems. It also features how to approximate higher order system using Dominant pole approximation. Furthermore it introduces the PID-controller



Authors:
- Jonas Holinger; jholinger@ethz.ch
- Shubham Gupta; shugupta@ethz.ch

## Learning Objectives

After completing this notebook, you should be able to:

1. Recognize the conditions when a steady state error occurs
2. Understand different Time-Domain specifications for first and second order system
3.Understand how to approximate higher order systems using Dominant pole approximation
4. Get a basic understanding of PID-Controllers


In [1]:
%pip install numpy matplotlib scipy ipywidgets control IPython sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.6/549.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.2 MB/s eta 0:00:00


In [2]:
import control as ct
import matplotlib.pyplot as plt
import numpy as np
import math
import ipywidgets as widgets
from scipy.integrate import odeint
from IPython.display import display, clear_output, Math
from ipywidgets import interactive, FloatSlider, VBox, ToggleButton, FloatSlider
from scipy import signal


# Steady State Error


The steady-state error is the difference between the system reference signal $r(t)$ and the closed-loop output $y(t)$ as $ \displaystyle \lim_{t \to \infty}$.

Applying the final value theorem, we know that we can calculate the steady-state error with:
<br><br>
$$ e_{ss}=  \displaystyle \lim_{t \to \infty } e(t) =  \displaystyle \lim_{s \to 0} s \cdot E(s) $$
### Example
We have an open-loop transfer function $C(s)P(s)$ with an integrator, and there is a pole at $s=0$. Let’s define the open-loop transfer function $C(s)P(s)$ as
$$ L(s) = \frac{K_{Bode}}{s(s+2)} $$<br>
where $K$ is our system's Bode gain, the $s$ term represents the system's integrator (making our system a **Type 1** system), and $(s+2)$ is an additional stable pole.

The input to the system is a first-order ramp ($q=1$) $r(t) = t$. This input corresponds in the frequency domain to:
$$ R(s) = \frac{1}{s^2} $$

To calculate the error signal $E(s)$, we use the sensitivity function, which is known from previous lectures. The sensitivity function maps the reference input $r$ to the error $e$:
$$ S(s) = \frac{1}{1+P(s)C(s)} = \frac{1}{1+L(s)} $$

We can multiply the sensitivity function $S$ with our input $R(s)$ and apply the final value theorem to get our steady-state error:
$$ e_{ss} = \displaystyle \lim_{s \to 0} s \cdot E(s) = \displaystyle \lim_{s \to 0} s \cdot R(s) \cdot \frac{1}{1+L(S)} = \displaystyle \lim_{s \to 0} s \cdot \frac{\frac{1}{s^2}}{1+\frac{K}{s(s+2)}} $$

The steady-state error we get after applying the limit is
$$ e_{ss} = \frac{2}{K} $$
which corresponds to the inverse of our Bode gain $K_{Bode}$. If the system is stable and $q=0$, the Bode gain corresponds to the steady-state gain for a unit step input and is called the DC gain.

If we now add a second integrator to our system, $C(s) = s^2$ making it a **Type 2** system and calculate our steady-state error again:
$$ e_{ss} = \displaystyle \lim_{s \to 0} s \cdot \frac{\frac{1}{s^2}}{1+\frac{K}{s^2(s+2)}} $$
we see that our steady-state error is exactly zero.

### Conclusion
To achieve a steady-state error of zero, you need to have one more integrator than the order of your ramp input. To achieve a small steady-state error, you can increase the Bode gain. To avoid calculating the steady-state error each time, you can use the following table:




| $ e_{ss} $ | $ q = 0 $                   | $ q = 1 $            | $ q = 2 $            |
|-------------|-----------------------------|----------------------|----------------------|
| Type 0      | $ \frac{1}{1 + K_{\text{Bode}}} $ | $ \infty $           | $ \infty $           |
| Type 1      | $ 0 $                        | $ \frac{1}{K_{\text{Bode}}} $ | $ \infty $           |
| Type 2      | $ 0 $                        | $ 0 $                | $ \frac{1}{K_{\text{Bode}}} $ |

### Visualization
Try different combinations of system types and input ramps and observe whether you have a steady-state error or not.


In [3]:
def simulate_system(system_type, input_type):

    t = np.linspace(0, 125, 1000)

    #P(s) = 1 / (s + 10)
    P = ct.TransferFunction([1], [1, 10])

    if system_type == "Type 0":
        C = ct.TransferFunction([5], [1])
    elif system_type == "Type 1":
        C = ct.TransferFunction([1], [1, 0])
    elif system_type == "Type 2":
        C = ct.TransferFunction([1], [1, 0, 0])



    open_loop_system = C * P
    closed_loop_system = ct.feedback(open_loop_system)

    if input_type == "Unit Ramp" and system_type == "Type 2":
        t = np.linspace(0, 1500, 1000)

    elif input_type == "Unit Parabola" and system_type == "Type 2":
        t = np.linspace(0, 300, 1000)


    if input_type == "Unit Step":
        input_signal = np.ones_like(t)
    elif input_type == "Unit Ramp":
        input_signal = t
    elif input_type == "Unit Parabola":
        input_signal = 0.5 * t**2


    t_out, y_out = ct.forced_response(closed_loop_system, T=t, U=input_signal)

    plt.figure(figsize=(12, 6))
    plt.plot(t, input_signal, 'r--', label='Input Signal', linewidth=2)
    plt.plot(t_out, y_out, 'b-', label='Output Response ', linewidth=2)



    plt.title(f'Input and Output of {system_type} System to {input_type} ')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.grid()
    plt.legend()
    plt.tight_layout()
    plt.show()



system_type_dropdown = widgets.Dropdown(
    options=["Type 0", "Type 1", "Type 2"],
    value="Type 0",
    description='System Type:',
)

input_type_dropdown = widgets.Dropdown(
    options=["Unit Step", "Unit Ramp", "Unit Parabola"],
    value="Unit Step",
    description='Input Type:',
)



interactive_plot = interactive(simulate_system,
                               system_type=system_type_dropdown,
                               input_type=input_type_dropdown)
display(interactive_plot)


interactive(children=(Dropdown(description='System Type:', options=('Type 0', 'Type 1', 'Type 2'), value='Type…

# Time Domain Specifications

##First-Order system

In both the time domain and the frequency domain, we can analyze a system's response. In the time domain, we typically separate the response into two stages: the transient stage and the steady-state stage. Now we want to examine different behaviors in our time domain.

Let’s consider a stable first-order system: <br>$$G(s)=\frac{K}{τs+1}$$<br>
For an inital condition of $x(0)=0$, the solution is given by: $$y(t)=K-e^{\frac{-t}{\tau}}$$.

We are interested in the following time-domain specifications for a stable first-order system:


**DC gain**
<br>
The Steady-state or DC gain **$y_{ss}$** is a measure of how much the systems output responds to an input after the transients effects have died out.
In our first order system the DC gain corresponds to K.

**Time constant**
<br>
The time constant **$\tau$** measures the speed of the system's response.
 It is defined as the time required for the system's unit step response to reach approximately 63% of its final value.

**Settling time**
<br>
The settling time **$T_d$** is the time it takes for the output to remain within $d\%$ of the final value. The settling time time can be approximated using the time constant $τ$ using the formula $$T_d=τ \cdot log(100/d)$$ For $5\%$ the approximation is $T_d=3\tau$


In [4]:
def plot_first_order_system(K=3, tau=2):

    sys = ct.TransferFunction([K], [tau, 1])

    t = np.linspace(0, 10 * tau, 1000)

    t, y = ct.step_response(sys, T=t)

    tau_time = tau
    yss = K
    settling_time = -np.log(0.05) * tau

    y_tau = K * (1 - np.exp(-tau_time / tau))
    y_settling = K * (1 - np.exp(-settling_time / tau))

    plt.figure(figsize=(10, 6))

    plt.plot(t, y, label="Step Response", color="blue")

    plt.plot(t, np.ones_like(t), label="Step Input", color="orange", linestyle='--')

    plt.axhline(yss, color="red", linestyle="--")

    plt.plot([tau_time, tau_time], [0, y_tau], color="green", linestyle="--")
    plt.plot([settling_time, settling_time], [0, y_settling], color="purple", linestyle="--")

    plt.text(tau_time, 0, f"τ    ", color="green", ha="center", va="bottom", fontsize=10, fontweight="bold")
    plt.text(settling_time, 0, f"Settling Time    ", color="purple", ha="center", va="bottom", fontsize=10, fontweight="bold")

    plt.text(0, yss, f"DC Gain", color="red", ha="left", va="bottom", fontsize=10, fontweight="bold")

    plt.xlabel("Time (s)")
    plt.ylabel("Response")
    plt.title(f"Step Response of First-Order System: G(s) = {K}/({tau}s + 1)")
    plt.legend(loc="best")
    plt.grid(True)
    plt.ylim(0, 1.2 * yss)
    plt.show()

K_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.1, description="K:")
tau_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.1, description="τ:")

interactive_plot = interactive(plot_first_order_system, K=K_slider, tau=tau_slider)
display(interactive_plot)


interactive(children=(FloatSlider(value=1.0, description='K:', max=10.0, min=0.1), FloatSlider(value=1.0, desc…

## Second-Order system

For a second-order system in the form:
$$G(s) = \frac{\omega_n^2}{s^2 + 2 \zeta \omega_n s + \omega_n^2}
\quad \Longleftrightarrow \quad
\begin{cases}
    \dot{x} = \begin{bmatrix} 0 & 1 \\ -\omega_n^2 & -2 \zeta \omega_n \end{bmatrix} x + \begin{bmatrix} 0 \\ \omega_n^2 \end{bmatrix} u \\
    y = x
\end{cases}
$$
We get the solution by imposing our zero initial condition and solving the algebra:

$$y(t) = 1 - \frac{1}{\cos \varphi} e^{\sigma t} \cos(\omega t + \varphi), \quad t \geq 0.$$

with $ϕ=arctan(\frac{σ}{ω})$

$\omega_n$ denotes the natural frequency of our system, sometimes also called the eigenfrequency. The natural frequency is defined as $\omega_n^2=\sigma^2+\omega^2$ and can be calculated from the pole of the system $s=\sigma+j\omega$.

The damping factor $\zeta = \frac{\sigma}{\omega_n}$ can be used to categorize our system into the following cases:

* $ζ<0$, underdamped
* $ζ=0$, critically damped
* $ζ>0$, overdamped

We can see that the location of the pole in our system determines the natural frequency, the damping ratio, and the time-domain specifications.


In [5]:
def plot_damping_responses(omega_n):
    damping_ratios = {
        "Underdamped (ζ < 1)": 0.5,
        "Critically Damped (ζ = 1)": 1.0,
        "Overdamped (ζ > 1)": 1.5
    }

    t = np.linspace(0, 10, 500)

    plt.figure(figsize=(12, 8))

    for label, zeta in damping_ratios.items():
        num = [omega_n**2]
        den = [1, 2*zeta*omega_n, omega_n**2]
        system = signal.TransferFunction(num, den)

        t, response = signal.step(system, T=t)

        plt.plot(t, response, label=label)

    plt.title('Step Responses of Second Order Systems')
    plt.xlabel('Time [s]')
    plt.ylabel('Response')
    plt.axhline(1, color='gray', linestyle='--', linewidth=0.8)
    plt.grid(True)
    plt.legend()
    plt.show()

omega_n_slider = FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='ωₙ')

interactive_plot = interactive(plot_damping_responses, omega_n=omega_n_slider)

display(interactive_plot)


interactive(children=(FloatSlider(value=1.0, description='ωₙ', max=10.0, min=0.1), Output()), _dom_classes=('w…

### Time-Domain Specifications for a second order system


**Steady-state Gain**

The Steady-state Gain remains the same with that for the first order system.
The time constant $\tau$ can be calculated using the formula:<br><br>  $$τ=\frac{1}{{|\sigma|}}$$.

**Settling Time**

The settling time $$T_d= \tau\cdot log(\frac{100}{d})$$<br> is also the same as in the first order system.
 For an overdamped system, where the poles are real and distinct, the settling time $T_d$ should be calculated using the pole with the largest time constant $\tau$. This pole represents the slowest decay rate, which dominates the system's response.


**Time to peak**

The time to peak is the duration it takes for the system response to reach its maximum value after a unit step input. For underdamped systems, it is given by:
$$T_p=\frac{\pi}{\omega}$$

**Hint:**
To calculate $\omega$ given natural frequency $w_n$ and damping coefficent $\zeta$ use  formula: $\omega=\omega_n\sqrt{1-\zeta^2}$


**Peak Overshoot Ratio**

The peak overshoot ratio is the maximum amount, by which the response exceeds the steady-state value. It can be expressed as:<br><br> $$M_p=e^{\frac{\sigma\pi}{\omega}}$$ <br> Alternatively, it can be related to the damping ratio $\zeta$ using: $\zeta^2=\frac{(lnM_p)^2}{\pi^2+(lnM_p)^2}$

**Rise Time**

The rise time is defined as the time required for the response to rise from 0% to 100% of the steady-state value.

$T_{100\%}=\frac{\frac{\pi}{2}-\phi}{\omega}\approx \frac{\pi}{2\omega_n}$

#### Visualisation
You can adjust the damping ratio and the natural frequency of the systems with the sliders. By pressing the buttons, you can display the different time-domain specifications. Observe how they change when you adjust the system's parameters.

In [18]:
def plot_step_response(omega_n, zeta, show_time_to_peak, show_peak_overshoot, show_rise_time, show_settling_time, show_time_constant):

    num = [omega_n**2]
    den = [1, 2*zeta*omega_n, omega_n**2]
    system = signal.TransferFunction(num, den)


    t = np.linspace(0, 20, 500)
    t, response = signal.step(system, T=t)

    # Time to Peak
    Tp = np.pi / (omega_n * np.sqrt(1 - zeta**2)) if zeta < 1 else np.nan

    # Peak Overshoot Ratio
    OS = np.exp(-zeta * np.pi / np.sqrt(1 - zeta**2)) * 100 if zeta < 1 else np.nan

    # Rise Time
    Tr = (1.8 / omega_n) if zeta < 0.69 else (np.nan if zeta > 1 else np.nan)

    # Settling Time
    Ts = 4 / (zeta * omega_n) if zeta > 0 else np.nan

    # Time Constant
    tau = 1 / (zeta * omega_n) if zeta > 0 else np.nan

    plt.figure(figsize=(20, 6))

    plt.subplot(1, 2, 1)
    plt.plot(t, response, label='Step Response')

    if show_time_to_peak and not np.isnan(Tp):
        plt.text(Tp, 0, f'Tp  ', color='red', fontsize=10, ha='right')
        plt.plot([Tp, Tp], [0, response[np.where(t >= Tp)[0][0]]], color='r', linestyle='--')

    if show_peak_overshoot:
        plt.text(Tp, response.max(), f'OS', color='red', fontsize=10, ha='center', va='bottom')

    if show_rise_time and not np.isnan(Tr):
        plt.text(Tr,0, f'T100%   ', color='green', fontsize=10, ha='right')
        plt.plot([Tr, Tr], [0, response[np.where(t >= Tr)[0][0]]], color='g', linestyle='--')

    if show_settling_time and not np.isnan(Ts):
        plt.text(Ts, 0, f'Td ', color='magenta', fontsize=10, ha='right')
        plt.plot([Ts, Ts], [0, response[np.where(t >= Ts)[0][0]]], color='m', linestyle='--')

    if show_time_constant and not np.isnan(tau):
        plt.text(tau, 0, f'τ ', color='cyan', fontsize=10, ha='right')
        plt.plot([tau, tau], [0, response[np.where(t >= tau)[0][0]]], color='c', linestyle='--')

    plt.title('Step Response of Second Order System')
    plt.xlabel('Time [s]')
    plt.ylabel('Response')
    plt.legend()
    plt.grid(True)
    plt.ylim(0, 1.1 * response.max())

    poles = np.roots(den)

    # Pole Plot
    plt.subplot(1, 2, 2)
    plt.scatter(np.real(poles), np.imag(poles), color='blue', marker='x')
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.title('Poles of the System')
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

omega_n_slider = FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='ωₙ')
zeta_slider = FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1, description='ζ')

show_time_to_peak_btn = ToggleButton(value=False, description='Time to Peak', button_style='info')
show_peak_overshoot_btn = ToggleButton(value=False, description='Peak Overshoot', button_style='info')
show_rise_time_btn = ToggleButton(value=False, description='Rise Time (T100%)', button_style='info')
show_settling_time_btn = ToggleButton(value=False, description='Settling Time', button_style='info')
show_time_constant_btn = ToggleButton(value=False, description='Time Constant', button_style='info')

interactive_plot = interactive(
    plot_step_response,
    omega_n=omega_n_slider,
    zeta=zeta_slider,
    show_time_to_peak=show_time_to_peak_btn,
    show_peak_overshoot=show_peak_overshoot_btn,
    show_rise_time=show_rise_time_btn,
    show_settling_time=show_settling_time_btn,
    show_time_constant=show_time_constant_btn
)

display(interactive_plot)


interactive(children=(FloatSlider(value=1.0, description='ωₙ', max=10.0, min=0.1), FloatSlider(value=0.5, desc…

### Constraints on Time-Domain Specifications

Imposing constraints on time-domain specifications restricts the permissible locations of system poles. For detailed derivations and formulas, refer to Problem Set 07.

**Settling Time**

Limiting the settling time $T_s$ constrains how far poles can be from the imaginary axis in the left-hand plane. A shorter settling time requires poles to be further to the left, indicating faster decay of the response.

**Peak Overshoot Ratio**

Restricting the peak overshoot ratio $M_p$ imposes a limit on the damping coefficient $\zeta$. Poles must lie within a cone defined by the angle $\theta = \arcsin(\zeta)$. A smaller overshoot requires a larger damping ratio, resulting in a narrower cone and poles closer to the real axis.

**Rise Time**

Limiting the rise time $T_{90}$ dictates a minimum natural frequency $\omega_n$ for the system. Since $\omega_n = \sqrt{\sigma^2 + \omega^2}$, the poles must lie outside a circle with radius $\omega_n$. A faster rise time necessitates a higher natural frequency, pushing the poles further away from the origin.

### Visualisation
Try calculating the different restrictions yourself and verify them against the plot. To satisfy the constraints, the pole must lie within the blue cone, outside the yellow circle, and to the left of the red line.

In [16]:
def plot_pole_cone(max_overshoot, T_90, T_s):

    zeta = -np.log(max_overshoot / 100) / np.sqrt(np.pi**2 + (np.log(max_overshoot / 100))**2)

    theta = np.arcsin(zeta)

    cone_angle = theta
    cone_slope = 1 / np.tan(cone_angle)

    real_parts = np.linspace(-200, 0, 400)

    imag_upper = cone_slope * (-real_parts)
    imag_lower = -imag_upper

    omega_n = (0.14 + 0.4 * zeta) * (2 * np.pi / T_90)

    sigma = -np.log(0.02) / T_s

    plt.figure(figsize=(10, 10))

    plt.fill_between(real_parts, imag_upper, imag_lower, color='lightblue', alpha=0.5, label=f'θ = {theta* (180 / 3.14):.2f}°')

    circle = plt.Circle((0, 0), omega_n, color='orange', fill=False, linestyle='--', label=f'ω_n >= {omega_n:.2f}')
    plt.gca().add_artist(circle)

    plt.axvline(x=-sigma, color='red', linestyle='--', label=f'σ <= -{sigma:.2f}')

    # Draw the axes
    plt.axhline(0, color='black', lw=0.5, ls='--')
    plt.axvline(0, color='black', lw=0.5, ls='--')

    plt.title(f'Permissible Pole Locations  (Max Overshoot = {max_overshoot}%, T90 = {T_90}s, Ts = {T_s}s)')
    plt.xlim(-70, 70)
    plt.ylim(-70,70 )
    plt.xlabel('Real Part')
    plt.ylabel('Imaginary Part')
    plt.grid()
    plt.legend()

    plt.show()

max_overshoot_slider = widgets.FloatSlider(value=10, min=0, max=100, step=1, description='Max Overshoot (%)')

T_90_slider = widgets.FloatSlider(value=0.05, min=0.01, max=1, step=0.01, description='Rise Time T90 (s)')

T_s_slider = widgets.FloatSlider(value=0.1, min=0.01, max=5, step=0.01, description='Settling Time Ts (s)')

interactive_plot = widgets.interactive(plot_pole_cone,
                                       max_overshoot=max_overshoot_slider,
                                       T_90=T_90_slider,
                                       T_s=T_s_slider)
display(interactive_plot)


interactive(children=(FloatSlider(value=10.0, description='Max Overshoot (%)', step=1.0), FloatSlider(value=0.…